In [6]:
print("""\
# *************************************************************************** #
# *************************************************************************** #
# TP2 : ALGEBRE LINEAIRE SUR UN ANNEAU PRINCIPAL                              #
# *************************************************************************** #
# *************************************************************************** #
""")

# CONSIGNES
#
# Les seules lignes a modifier sont annoncee par "Code pour l'exercice"
# indique en commmentaire et son signalees
# Ne changez pas le nom des variables
#
# CONSEILS
#
# Ce modele vous sert a restituer votre travail. Il est deconseille d'ecrire
# une longue suite d'instruction et de debugger ensuite. Il vaut mieux tester
# le code que vous produisez ligne apres ligne, afficher les resultats et
# controler que les objets que vous definissez sont bien ceux que vous attendez.
#
# Vous devez verifier votre code en le testant, y compris par des exemples que
# vous aurez fabrique vous-meme.
#


reset()
print("""\
# ****************************************************************************
# MISE SOUS FORME NORMALE D'HERMITE
# ****************************************************************************
""")

# Donnees de l'enonce de l'exercice

A = matrix(ZZ,[
        [-2,  3,  3,  1],
        [ 2, -1,  1, -3],
        [-4,  0, -1, -4]])

A1 = random_matrix(ZZ, 7, 8, algorithm='echelonizable', rank=3)

U = identity_matrix(4)

# Code pour l'EXERCICE

def cherche_pivot_non_nul(A, U, i, j):
    """
    Echange la colonne j avec une colonne à gauche pour que A[i,j] soit non nul.
    """
    """
    The function aims to find the rightmost column (denoted as j0)
     in the submatrix A[:, j:] (columns from j to the end) 
     such that the element at position (i, j0) is non-zero. 

    If such a column is found (i.e., j0 != -1), 
    it swaps columns j and j0 in both matrices A and U. 
    The function returns True if a pivot is found and False otherwise.

    In the context of Hermite Normal Form computations, 
    this function is likely used to ensure that 
    the leading entry in the current row (specified by the index i) is non-zero, 
    facilitating subsequent operations in the reduction process.
    """
    j0 = -1
    for k in range(j,-1,-1):
        if A[i,k]!=0:
            j0 = k
            break
    
    if j0 != -1:
       A.swap_columns(j, j0)
       U.swap_columns(j, j0)
       return True #returns True if a pivot has been found
    else:
        return False
    

def normalise_pivot(A, U, i, j):
    """
    Multiplie la colonne j si besoin pour que A[i,j] soit positif.
    """
    """
    The purpose of this function is to ensure that the element at position (i,j)
    un matrix A is positive.
    This function checks if the element at position (i,j) in matrix A
    is negative. The function multiplies the entire column j by -1 to make it positive.
    """
    if A[i,j] < 0:
        A[:,j] = A[:,j] * -1
        U[:,j] = U[:,j] * -1

def annule_a_gauche(A, U, i, j):
    """
    Annule les coefficients à gauche de A[i,j]
    Eliminate the coefficients to the left of the element A[i,j]
    in matrix A and U.
    it performs operations to eliminate the coefficient 
    at position (i, k) using Bézout's relations. 
    """
    for k in range(j-1, -1, -1):
        a = A[i,j]
        b = A[i,k]
        d, u, v = xgcd(a, b) #Eucidean
        A[:,j], A[:,k] = u * A[:,j] + v * A[:,k], -(b/d) * A[:,j] + (a/d) * A[:,k]
        U[:,j], U[:,k] = u * U[:,j] + v * U[:,k], -(b/d) * U[:,j] + (a/d) * U[:,k]

def reduit_a_droite(A,U,i,j):
    """
    Réduit les coefficients à gauche de A[i,j] modulo A[i,j]
    subtracting a multiple of the j-th column from the k-th column 
    to make the coefficient at (i,k) modulo 'A[i,j]'
    """
    for k in range(j+1, A.ncols()):
        quotient  = A[i,k]// A[i,j]
        A[:,k] = A[:,k] - quotient * A[:,j]
        U[:,k] = U[:,k] - quotient * U[:,j]



def MyHNF(A):
    """
    Forme normale d'Hermite selon votre code
    """
    m = A.nrows()
    n = A.ncols()
    H = copy(A)
    U = identity_matrix(n)
    l = 0 # initialize a variable l to 0
    i = m-1 # start from the bottom row
    k = n-1 # start from the rightmost column

    while i>= l:
        # Check if there is a non-zero pivot in the current column
        if cherche_pivot_non_nul(H, U, i, k):
            # If a pivot is found, perform the following operations:
            normalise_pivot(H,U,i,k) #Normalize the pivot element
            annule_a_gauche(H,U,i,k) #Make entries above the pivot zero
            reduit_a_droite(H,U,i,k) #Make entries to the right of the pivot zero
        
            i = i-1
            k = k-1
        else:
            i = i-1
    # Check if HNF computation is successful
    assert(H-A*U==0)
    # Return the computed Hermite Normal Form (H) and the transformation matrix (U)
    return H,U

def SageHNF(A):
    """
    Forme normale d'Hermite de SAGE avec la convention francaise :
    Les vecteurs sont ecrits en colonne.
    """
    m = A.nrows()
    n = A.ncols()
    Mm = identity_matrix(ZZ,m)[::-1,:]
    Mn = identity_matrix(ZZ,n)[::-1,:]
    AA = (Mm*A).transpose()
    HH, UU = AA.hermite_form(transformation=True)
    H = (HH*Mm).transpose()*Mn
    U = UU.transpose()*Mn
    assert(H-A*U==0)
    return H,U

H,  U  = MyHNF(A)
HH, UU = SageHNF(A)

# test number matrix
def test(number):
    for i in range(number):
        n = ZZ.random_element(20)
        m = ZZ.random_element(20)

        A = random_matrix(ZZ,n,m)
        H, U = MyHNF(A)
        HH, UU = SageHNF(A)
        if HH!=H:
            return False
    return True

test = test(10)

# # Affichage des resultats

print("\n$ Question 4")
print("La matrice A = ")
print(A)
print("a pour forme normale d'Hermite H=")
print(H)
print("et matrice de transformation U=")
print(U)
print("\n$ Question 5")
print("D'apres SageMath, la matrice A a pour forme normale d'Hermite H=")
print(HH)
print("et matrice de transformation U=")
print(UU)
print("\n$ Question 6")
print("Les deux fonctions coincident-elles sur une centaine d'exemples ?")
print(test)

# *************************************************************************** #
# *************************************************************************** #
# TP2 : ALGEBRE LINEAIRE SUR UN ANNEAU PRINCIPAL                              #
# *************************************************************************** #
# *************************************************************************** #

# ****************************************************************************
# MISE SOUS FORME NORMALE D'HERMITE
# ****************************************************************************


$ Question 4
La matrice A = 
[-2  3  3  1]
[ 2 -1  1 -3]
[-4  0 -1 -4]
a pour forme normale d'Hermite H=
[0 4 1 2]
[0 0 1 0]
[0 0 0 1]
et matrice de transformation U=
[  8   3   3   6]
[ 19   8   7  15]
[-12  -4  -4  -9]
[ -5  -2  -2  -4]

$ Question 5
D'apres SageMath, la matrice A a pour forme normale d'Hermite H=
[0 4 1 2]
[0 0 1 0]
[0 0 0 1]
et matrice de transformation U=
[  8   3   3   6]
[ 19 